In [7]:
import numpy as np  # Importa a biblioteca numpy para operações numéricas
import pandas as pd  # Importa a biblioteca pandas para manipulação de dados tabulares
from scipy.optimize import fsolve  # Importa fsolve do scipy para resolver equações não lineares

# Carregamento de Dados
# VentoCurvaPotencia.txt contém dados de velocidade do vento e potência eólica
# Pd2022a2031.txt contém dados de demanda horária da rede de 2022 a 2031
# Brava_consumo.xlsx contém dados de consumo de energia de um arquivo Excel
vento_curva_potencia = np.loadtxt('VentoCurvaPotencia.txt')  # Carrega os dados de vento e potência
dados_consumo = pd.read_excel('Brava_consumo.xlsx', sheet_name='Carga2010-2037')  # Lê os dados de consumo

# Definição de Cenários
cen = np.zeros((7, 4))  # Inicializa a matriz de cenários
cen[0, :] = [150, 2, 2, 1500]  # Define o primeiro cenário
cen[1, :] = [300, 1, 3, 1500]  # Define o segundo cenário
# Define os próximos cenários...

# Inicialização de Matrizes de Resultados
res = np.zeros((cen.shape[0], 14))  # Inicializa a matriz de resultados
res2 = np.zeros((cen.shape[0], 8))  # Inicializa outra matriz de resultados

# Loop Principal (para cada cenário)
for k in range(cen.shape[0]):
    ano = 20  # anos do projeto
    RS = cen[k, 3]  # Capacidade do reservatório em m3
    ERmax = RS / 3600 * 9.8 * 110  # Capacidade do reservatório em kWh
    h = 8760  # Quantidade de horas de simulação
    Er = np.zeros((h, ano))  # Inicialização do vetor-energia do reservatório
    Erj = ERmax / 2  # Energia inicial no reservatório
    Pt = np.zeros((h, ano))  # Potência hidráulica
    Pb = np.zeros((h, ano))  # Potência de bombeamento
    Erej = np.zeros((h, ano))  # Energia rejeitada
    Pge = np.zeros((h, ano))  # Potência de geração elétrica
    Pwr = np.zeros((h, ano))  # Potência eólica
    nb = 0.86  # Eficiência média das bombas
    nt = 0.86  # Eficiência média das turbinas hidráulicas
    nw = 0.97  # Eficiência elétrica da conversão da potência eólica
    Te = cen[k, 0]  # Potência unitária do aerogerador
    ne = cen[k, 1]  # Número de aerogeradores
    PDmin = 256 * 0.5  # Potência mínima do grupo eletrogéneo

    # Cálculo da potência eólica
    Pwt = ne * nw * vento_curva_potencia[:h, int(cen[k, 2]) - 1]
    Ptmax = np.mean(Pwt)  # Potência máxima da turbina hidráulica

    # Loop para cada ano
    for i in range(ano):
        # Loop para cada hora no ano
        for j in range(h):
            # Simulação da produção eólica e bombeamento
            if Pwt[j] > dados_consumo.values[j, i] * 0.35:
                Pwr[j, i] = dados_consumo.values[j, i] * 0.35
                if (Pwt[j] - Pwr[j, i]) * nb < (ERmax - Erj):
                    Pb[j, i] = Pwt[j] - Pwr[j, i]
                    Erj = Erj + (Pwt[j] - Pwr[j, i]) * nb
                else:
                    Pb[j, i] = (ERmax - Erj) / nb
                    Erj = ERmax
                    Erej[j, i] = (Pwt[j] - Pwr[j, i]) - (ERmax - Erj) / nb
            else:
                Pwr[j, i] = Pwt[j]

            # Simulação da energia de compensação hidráulica ou diesel
            if (dados_consumo.values[j, i] - Pwr[j, i]) <= Erj * nt:
                if (dados_consumo.values[j, i] - Pwr[j, i]) <= Ptmax:
                    Pt[j, i] = dados_consumo.values[j, i] - Pwr[j, i]
                    Erj = Erj - Pt[j, i] / nt
                elif (dados_consumo.values[j, i] - Pwr[j, i]) > Ptmax and \
                        (dados_consumo.values[j, i] - Pwr[j, i]) <= (Ptmax + PDmin):
                    Pge[j, i] = PDmin
                    Pt[j, i] = (dados_consumo.values[j, i] - Pwr[j, i]) - Pge[j, i]
                    Erj = Erj - Pt[j, i] / nt
                elif (dados_consumo.values[j, i] - Pwr[j, i]) > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = dados_consumo.values[j, i] - Pwr[j, i] - Pt[j, i]
                    Erj = Erj - Ptmax / nt
            elif (dados_consumo.values[j, i] - Pwr[j, i]) > Erj * nt:
                if Erj * nt > Ptmax and Erj * nt < (Ptmax + PDmin):
                    if Ptmax >= PDmin and (dados_consumo.values[j, i] - Pwr[j, i]) <= Ptmax:
                        Pge[j, i] = PDmin
                        Pt[j, i] = (dados_consumo.values[j, i] - Pwr[j, i]) - PDmin
                        Erj = Erj - Pt[j, i] / nt
                    else:
                        Pge[j, i] = dados_consumo.values[j, i] - Pwr[j, i]
                elif Erj * nt > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = dados_consumo.values[j, i] - Pwr[j, i] - Pt[j, i]
                    Erj = Erj - Ptmax / nt
                else:
                    Pge[j, i] = dados_consumo.values[j, i] - Pwr[j, i]

            Er[j, i] = Erj

    # Cálculo de Resultados
    # Cálculos adicionais e análises de resultados são realizados aqui

    # Análise Financeira
    # Avaliações financeiras são feitas e resultados são armazenados


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.